In [2]:
import torch 
import torch.autograd as autograd
import numpy as np 
import torch.nn as nn
from load_data import *
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim


In [3]:
## Loading data 

embedding_dict = load_embedding_dict()
sentence_Y_dict = load_sentence_Y_dict()

print "Loading batch dicts"
train_batch_dict_new = load_train_data()
validation_batch_dict_new = load_validation_data()


Loading batch dicts


In [4]:
corpus_word_vocab = load_corpus_vocab()

In [5]:
vocab_size = len(corpus_word_vocab)
embedding_dim = 300
hidden_dim = 100
batch_size = 64 
n_classes = 5

In [6]:
embedding_lookup  = nn.Embedding(vocab_size,embedding_dim,padding_idx = 0)

In [7]:
bilstm = nn.LSTM(embedding_dim,hidden_dim, bidirectional = True)
linear_layer = nn.Linear(2*hidden_dim,n_classes)


In [8]:
class BiLSTM(nn.Module) :
    
    def __init__(self,embedding_dim,hidden_dim,vocab_size,n_classes) :
        
        super(BiLSTM,self).__init__()
        self.embedding_dim = embedding_dim 
        self.hidden_dim = hidden_dim 
        self.vocab_size = vocab_size
        self.n_classes = n_classes
        
        self.embedding_lookup = nn.Embedding(vocab_size,embedding_dim,padding_idx = 0)
        self.bilstm = nn.LSTM(embedding_dim,hidden_dim, bidirectional = True)
        self.linear_layer = nn.Linear(2*hidden_dim,n_classes)
        self.hidden = self.init_hidden(hidden_dim)
        
        
    def forward(self,sentence,embedding_dict,s_id) :
        
        embeds = self.embedding_lookup(sentence)
       
        #print "embedding dimension",embeds.view(len(embedding_dict[s_id]),1,embedding_dim).size()

        lstm_out, self.hidden = self.bilstm(embeds.view(len(embedding_dict[s_id]),1,embedding_dim),self.hidden)
        lstm_out = lstm_out.view(len(embedding_dict[s_id]),2*hidden_dim)

        output_vec = self.linear_layer(lstm_out)

        return output_vec
    def init_hidden(self,hidden_dim) :
        return (Variable(torch.randn(2,1,hidden_dim)),Variable(torch.randn(2,1,hidden_dim)))
        
        

In [9]:
def convert_to_onehot(batch_labels) :


	### Label -1 assigned to the pad values and before input and mapped  to [0 0 0 0 0]
	output_mat = []
	for sent_labels in batch_labels :
		temp_mat = []	
		for word_label in sent_labels :
			if word_label == -1:

				temp = [0]*5
				temp_mat.append(temp)
				continue
			temp = [0]*5
			temp[word_label] = 1
			temp_mat.append(temp)

		output_mat.append(temp_mat)

	return np.asarray(output_mat)

def pad_zeros(ids,embedding_dict) :
	## return a numpy array of all the sentences in the batch padded with zeros for their embeddings 

	max_len = 0 
	for sent_id in ids :
		embedding = embedding_dict[sent_id]
		if len(embedding) > max_len :
			max_len = len(embedding)

	#print "max_len", max_len


	sent_mat = [embedding_dict[sent_id] for sent_id in ids]

	if len(np.array(sent_mat).shape) == 1 :
			return np.array([xi + [0]*(max_len - len(xi)) for xi in sent_mat],dtype='int'),max_len

	else :

		padded_mat = np.zeros((len(ids),max_len),dtype='int')

		sent_mat = np.array(sent_mat,dtype= 'int')
		for i in xrange(len(ids)) :
	 		padded_mat[i,:] = np.pad(sent_mat[i,:],(0,max_len - sent_mat[i,:].shape[0]),'constant',constant_values = (0))


		return padded_mat,max_len

def test_on_validation_data(model,sentence_Y_dict, validation_batch_dict_new,embedding_dict,use_whole = False) :

	batch_count = 0 
	validation_loss = 0 
	acc  = 0 
	for batch_n,sent_ids in validation_batch_dict_new.iteritems() :

		vList = []
		batch_count +=1 

		sents , max_len = pad_zeros(sent_ids,embedding_dict)

		labels = [sentence_Y_dict[item]+[-1]*(max_len - len(sentence_Y_dict[item])) for item in sent_ids] 
				
		labels = convert_to_onehot(labels)

		vList = model.test_on_batch(sents,labels)
		validation_loss+= vList[0]
		acc+=vList[1]

		if batch_count == 30 and use_whole == False :
			break 


	print "validation_loss :",validation_loss/batch_count
	acc = acc/batch_count 
	print "Accuracy :",acc
	return [validation_loss/batch_count,acc]


In [10]:
## Preparing data for input 
epochs = 1
loss_function = nn.CrossEntropyLoss()
## Going over each sentence at a time 

batch_size = 64

model = BiLSTM(embedding_dim,hidden_dim,vocab_size,n_classes)
optimizer = torch.optim.RMSprop(model.parameters(),lr = 0.0001)
def intialise_hidden(hidden_dim) :
    
    return (Variable(torch.randn(2,1,hidden_dim)),Variable(torch.randn(2,1,hidden_dim)))

for i in range(epochs) :
    batch_count = 0
    
    for batch,sent_ids in train_batch_dict_new.iteritems() :
        batch_count+= 1
        s_count = 0 
        
        sent_loss = Variable(torch.FloatTensor(len(sent_ids))) ## len(sent_ids)
        for s_id in sent_ids :
            
            model.zero_grad()
            #hidden = intialise_hidden(hidden_dim)
            
            model.hidden = model.init_hidden(hidden_dim)
            
            sentence = Variable(torch.LongTensor(embedding_dict[s_id]))
            labels = sentence_Y_dict[s_id]

            targets = Variable(torch.LongTensor(labels))
            #print "target_size", targets.size()
            
            output_vec = model.forward(sentence,embedding_dict,s_id)
            """embeds = embedding_lookup(sentence)
       
            #print "embedding dimension",embeds.view(len(embedding_dict[s_id]),1,embedding_dim).size()
        
            lstm_out, hidden = bilstm(embeds.view(len(embedding_dict[s_id]),1,embedding_dim),hidden)
            lstm_out = lstm_out.view(len(embedding_dict[s_id]),2*hidden_dim)

            output_vec = linear_layer(lstm_out)
            """
            # output_vec --> sentence_length * n_classes
            
            
            #if s_count == 10 :
                #break 
            sent_loss[s_count] = loss_function(output_vec,targets)
            s_count+=1 
            
        ## One batch finished 
        avg_batch_loss = sum(sent_loss)/batch_size
        print avg_batch_loss
        avg_batch_loss.backward()
        optimizer.step()
        
        if batch_count ==10 :
            break 

Variable containing:
 1.6305
[torch.FloatTensor of size 1]

Variable containing:
 1.5661
[torch.FloatTensor of size 1]

Variable containing:
 1.5297
[torch.FloatTensor of size 1]

Variable containing:
 1.4867
[torch.FloatTensor of size 1]

Variable containing:
 1.4570
[torch.FloatTensor of size 1]

Variable containing:
 1.4131
[torch.FloatTensor of size 1]

Variable containing:
 1.3745
[torch.FloatTensor of size 1]

Variable containing:
 1.3553
[torch.FloatTensor of size 1]

Variable containing:
 1.3222
[torch.FloatTensor of size 1]

Variable containing:
 1.3221
[torch.FloatTensor of size 1]



In [11]:
model.parameters()

<generator object parameters at 0x7f746cf38640>

In [12]:
batch_loss

NameError: name 'batch_loss' is not defined

In [15]:
# **Writing the CRF Model along with a BiLSTM**

In [18]:
## Defining the model here 
vocab_size = len(corpus_word_vocab)
embedding_dim = 300
hidden_dim = 100
batch_size = 64 
n_classes = 7 

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (autograd.Variable(torch.randn(2, 1, self.hidden_dim)),
                autograd.Variable(torch.randn(2, 1, self.hidden_dim)))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.Tensor(1, self.tagset_size).fill_(-10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = autograd.Variable(init_alphas)

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward variables at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = autograd.Variable(torch.Tensor([0]))
        tags = torch.cat([torch.LongTensor([self.tag_to_ix[START_TAG]]), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.Tensor(1, self.tagset_size).fill_(-10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = autograd.Variable(init_vvars)
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id])
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq
    
    

In [ ]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 300
HIDDEN_DIM = 100

## get training data here
n_classes = 7
tag_to_idx  = {"B-skill": 0, "I-exp": 1, "O": 2, "I-skill": 3, "B-exp": 4,START_TAG : 5,STOP_TAG :6}

model = BiLSTMCRF(embedding_dim,hidden_dim,vocab_size,n_classes)

for i in xrange(epochs) :
    
    for batch,sent_ids in train_batch_dict_new :
        
        for s_id in sent_ids :
            
            sentence = Variable(torch.LongTensor(embedding_dict[s_id]))
            labels = sentence_Y_dict[s_id]
            
            score,tag_sequence = model.forward(sentence,s_id,embedding_dict)
            
            
            ## Score of the best sequence
            
           
            